# Sesión 2: Lectura de Datos y Análisis con Pandas en Ciencias Ambientales

## Parte 1: Lectura de Archivos y Consumo de APIs

### 1.1 Lectura de Archivos CSV


In [ ]:
import pandas as pd

In [ ]:
### 1.2 Lectura de Archivos Excel

# Leer un archivo Excel con datos meteorológicos
df_weather = pd.read_excel('/Users/gonzalezulises/Documents/GitHub/101_Data_Analytics_Rizoma/data/Pronostico_del_tiempo.xlsx', sheet_name='Hoja 1 - data (1)')
print(df_weather.head())

In [ ]:

# Leer un archivo Excel con datos meteorológicos
df_weather = pd.read_excel('/Users/gonzalezulises/Documents/GitHub/101_Data_Analytics_Rizoma/data/Pronostico_del_tiempo.xlsx', sheet_name='Hoja 1 - data (1)')
print(df_weather.head())

In [ ]:
### 1.3 Consumo de API

import requests

# Obtener datos de calidad del aire de una API
api_url = "https://api.openaq.org/v1/latest?country=ES&parameter=pm25"
response = requests.get(api_url)
air_quality_data = response.json()


In [ ]:

# Convertir los datos de la API a un DataFrame
df_air_quality = pd.DataFrame(air_quality_data['results'])
print(df_air_quality.head())


## Parte 2: Análisis de Datos con Pandas

In [ ]:
# Leer un archivo CSV con datos de calidad del agua
df_water_quality = pd.read_csv('https://raw.githubusercontent.com/gonzalezulises/101_Data_Analytics_Rizoma/master/data/water_potability.csv')
print(df_water_quality.head())

### 2.1 Exploración Inicial de Datos


In [ ]:

# Usando el DataFrame de calidad del agua
print(df_water_quality.info())

In [ ]:
print(df_water_quality.describe())

In [ ]:
# Verificar valores nulos
print(df_water_quality.isnull().sum())

In [ ]:
### 2.2 Selección y Filtrado de Datos

# Seleccionar columnas específicas
ph_and_temp = df_water_quality[['pH', 'Temperature']]

In [ ]:
# Filtrar datos
high_ph = df_water_quality[df_water_quality['pH'] > 8]

In [ ]:
### 2.3 Agrupación y Agregación

# Agrupar por ubicación y calcular promedios
avg_by_location = df_water_quality.groupby('Location').mean()
print(avg_by_location)

In [ ]:
## 2.4 Operaciones con Series y DataFrames

# Añadir una nueva columna
df_water_quality['pH_category'] = pd.cut(df_water_quality['pH'], 
                                         bins=[0, 6.5, 7.5, 14],
                                         labels=['Acidic', 'Neutral', 'Alkaline'])

In [ ]:
df_water_quality

In [ ]:
df_weather.head()

In [ ]:
# Operaciones entre columnas
df_weather['diferencia_temperatura'] = df_weather['Temperatura Máxima'] - df_weather['Temperatura Mínima']
df_weather.head()

In [ ]:
### 2.5 Manejo de Datos Faltantes


In [ ]:
df_water_quality.info()

In [ ]:
# Verificar si hay valores NaN en el DataFrame
tiene_NaN = df_water_quality.isnull().values.any()
print(f"¿El DataFrame tiene valores NaN?: {tiene_NaN}")

In [ ]:
# Contar el número de valores NaN en cada columna
NaN_por_columna = df_water_quality.isnull().sum()
print("Número de valores NaN en cada columna:")
print(NaN_por_columna)

In [ ]:
# Calcular el número total de filas
total_columnas = df_water_quality.shape[0]
total_columnas

In [ ]:
# Calcular el porcentaje de NaN en cada columna
porcentaje_NaN= ((NaN_por_columna / total_columnas) * 100).round(2)
print("Porcentaje de valores NaN en cada columna:")
print(porcentaje_NaN)

# ¿Cómo se manejan los datos faltantes en un análisis estadístico?

El manejo de datos faltantes es crucial en el análisis estadístico. Cuando los valores no están disponibles (por ejemplo, registros incompletos o errores), existen estrategias para abordarlos:

- Eliminar filas con datos faltantes:
Si la cantidad de datos faltantes es pequeña, puedes eliminar las filas correspondientes. Sin embargo, esto puede reducir el tamaño del conjunto de datos.
- Imputación (relleno) de valores:
  - Media o mediana: Reemplaza los valores faltantes por la media o mediana de la variable.
  - Moda: Usa la moda (valor más común) para imputar datos categóricos.
  - Imputación múltiple: Crea múltiples conjuntos de datos imputados y combínalos para reducir el sesgo.
- Técnicas avanzadas:
  - Modelos de aprendizaje automático: Utiliza algoritmos para predecir valores faltantes basándose en otras variables.
  - Imputación basada en reglas: Define reglas específicas para imputar valores.

# ¿Hay algún criterio estadístico?

No existe un umbral específico universalmente aceptado para el porcentaje máximo de datos faltantes en una columna. La decisión depende del contexto y del rol de la variable en el análisis. Algunos puntos a considerar:

+ Importancia de la variable:
  - Si la columna representa una variable crítica o una respuesta en un modelo, incluso un pequeño porcentaje de datos faltantes puede ser problemático.
  - Si es una variable menos relevante, puedes ser más tolerante con los valores faltantes.
+ Tipo de análisis:
  - En análisis exploratorio, se permite más flexibilidad. Sin embargo, en modelos predictivos o inferenciales, los datos faltantes pueden afectar los resultados.
  - En estudios observacionales, se puede aceptar un mayor porcentaje de datos faltantes que en ensayos controlados.
+ Tamaño de la muestra:
  - Si el conjunto de datos es grande, un porcentaje bajo de datos faltantes puede ser manejable.
  - En muestras pequeñas, incluso un pequeño porcentaje puede afectar la validez.
+ Técnicas de imputación:
  - Si puedes imputar valores faltantes de manera confiable (por ejemplo, mediante imputación múltiple), puedes ser más flexible.

# Eliminación de Filas o Columnas

### Duplicar el dataframe


In [ ]:
df_water_quality_copy = df_water_quality.copy()

In [ ]:
df_water_quality_copy

### Eliminar una Columna

Para eliminar una columna del DataFrame, utilizaremos el método `drop()`. Asegúrate de especificar el argumento `axis=1` para indicar que estás eliminando una columna y, opcionalmente, puedes usar `inplace=True` si deseas modificar el DataFrame directamente.

In [ ]:
df_water_quality_copy.drop('Sulfate', axis=1, inplace=True)

###  Imputación Simple con la media
Reemplazaremos los valores NaN con la media de la columna.

In [ ]:
df_water_quality_copy['ph'].fillna(df_water_quality_copy['ph'].mean(), inplace=True)

In [ ]:
df_water_quality_copy.isnull().sum()

### Imputación Simple con la mediana

In [ ]:
df_water_quality_copy['Trihalomethanes'].fillna(df_water_quality_copy['Trihalomethanes'].median(), inplace=True)

In [ ]:
df_water_quality_copy.isnull().sum()

### Imputación Simple con la moda

Reemplazar valores NaN con la moda de la columna (más común para datos categóricos).

In [ ]:
df_water_quality_copy['ph_category'].fillna(df_water_quality_copy['ph_category'].mode()[0], inplace=True)

In [ ]:
df_water_quality_copy.isnull().sum()

## Ejercicios Prácticos

- Carga un conjunto de datos de precipitaciones mensuales desde el mi repositorio github (https://raw.githubusercontent.com/gonzalezulises/detodounpoco/main/PREC_2021_Provincias.csv) y resuelve los siguientes ejercicios:

1. Calcula la precipitación anual por región y muestra las 5 regiones más lluviosas. Usa `groupby()` para agrupar por región, `sum()` para sumar precipitaciones, `sort_values()` para ordenar y `head()` para mostrar las top 5.

2. Encuentra los 3 meses más lluviosos y los 3 más secos en promedio para toda España. Utiliza `melt()` para reestructurar el DataFrame, `groupby()` para agrupar por mes, `mean()` para calcular promedios, `sort_values()` para ordenar y `head()` para seleccionar los extremos.

3. Agrupa las regiones por niveles de precipitación anual (bajo: <500mm, medio: 500-1000mm, alto: >1000mm). Emplea `cut()` para categorizar las precipitaciones y `value_counts()` para contar las regiones en cada categoría.

4. Crea un DataFrame con la variación mensual de precipitaciones para las 5 regiones más lluviosas. Usa `set_index()` para establecer las regiones como índice, `nlargest()` para seleccionar las top 5 regiones basadas en precipitación anual, y `T` para transponer el resultado si es necesario.

5. Identifica regiones con condiciones extremas: a) al menos un mes >200mm, b) al menos tres meses consecutivos <10mm. Utiliza `melt()` para reestructurar, `groupby()` con `any()` para identificar meses >200mm, y `rolling()` con `all()` para periodos consecutivos <10mm.

6. Calcula la correlación entre la precipitación de verano (junio, julio, agosto) y la anual. Selecciona las columnas relevantes con `[]` y usa `corr()` para calcular la correlación.

7. Prepara un DataFrame para un mapa de calor de precipitaciones mensuales por región. Emplea `set_index()` para establecer las regiones como índice y selecciona solo las columnas de los meses con `[]`.

8. Analiza la estacionalidad calculando el porcentaje de precipitación por estación para cada región. Agrupa los meses por estaciones usando `[]`, luego aplica `groupby()` y `sum()` para totales estacionales, y `apply()` con una función lambda para calcular porcentajes.

9. Compara precipitaciones entre regiones costeras e interiores. Usa `isin()` para clasificar regiones como costeras o interiores, `groupby()` para agrupar por tipo, y `agg()` para calcular estadísticas descriptivas por grupo.

10. Identifica posibles anomalías en los datos de precipitación mensual. Utiliza `melt()` para reestructurar, `groupby()` con `quantile()` para calcular umbrales de anomalías, y `query()` para filtrar valores atípicos basados en estos umbrales.



## Soluciones

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/gonzalezulises/detodounpoco/main/PREC_2021_Provincias.csv')

In [ ]:
# 1. Precipitación anual por región y top 5 más lluviosas
precipitacion_anual = df.groupby('region')['anual'].sum().sort_values(ascending=False)
print("Top 5 regiones más lluviosas:")
print(precipitacion_anual.head())


In [ ]:

# 2. 3 meses más lluviosos y 3 más secos en promedio
# Identificar las columnas mensuales
meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

# Calcular el promedio de precipitación para cada mes
precipitacion_mensual = df[meses].mean().sort_values(ascending=False)

# Mostrar los 3 meses más lluviosos
print("3 meses más lluviosos:")
print(precipitacion_mensual.head(3))

# Mostrar los 3 meses más secos
print("\n3 meses más secos:")
print(precipitacion_mensual.tail(3))

# Si quieres ver todos los meses ordenados
print("\nTodos los meses ordenados por precipitación:")
print(precipitacion_mensual)


In [ ]:

# 3. Agrupar regiones por niveles de precipitación anual
df['categoria_precipitacion'] = pd.cut(df['anual'], 
                                       bins=[0, 500, 1000, float('inf')], 
                                       labels=['Bajo', 'Medio', 'Alto'])
conteo_categorias = df['categoria_precipitacion'].value_counts()
print("\nConteo de regiones por categoría de precipitación:")
print(conteo_categorias)


In [ ]:

# 4. Variación mensual de precipitaciones para las 5 regiones más lluviosas
top_5_regiones = df.nlargest(5, 'anual')['region'].tolist()
df_top_5 = df[df['region'].isin(top_5_regiones)].set_index('region')
df_top_5_mensual = df_top_5.loc[:, 'enero':'diciembre']
print("\nVariación mensual de precipitaciones para las 5 regiones más lluviosas:")
print(df_top_5_mensual)


In [ ]:

# 5. Identificar regiones con condiciones extremas
df_melt = df.melt(id_vars=['Parametro', 'region', 'anual'], 
                  var_name='mes', value_name='precipitacion')
regiones_extremas = df_melt.groupby('region').agg(
    max_precipitacion=('precipitacion', 'max'),
    meses_secos=('precipitacion', lambda x: (x < 10).rolling(3).sum().max())
)
regiones_extremas = regiones_extremas[
    (regiones_extremas['max_precipitacion'] > 200) | 
    (regiones_extremas['meses_secos'] >= 3)
]
print("\nRegiones con condiciones extremas:")
print(regiones_extremas)


In [ ]:

# 6. Correlación entre precipitación de verano y anual
df['precipitacion_verano'] = df[['junio', 'julio', 'agosto']].sum(axis=1)
correlacion = df['precipitacion_verano'].corr(df['anual'])
print(f"\nCorrelación entre precipitación de verano y anual: {correlacion:.2f}")


In [ ]:

# 7. Preparar DataFrame para mapa de calor
df_heatmap = df.set_index('region').loc[:, 'enero':'diciembre']
print("\nDataFrame para mapa de calor de precipitaciones mensuales:")
print(df_heatmap.head())


In [ ]:

# 8. Analizar estacionalidad
df['primavera'] = df[['marzo', 'abril', 'mayo']].sum(axis=1)
df['verano'] = df[['junio', 'julio', 'agosto']].sum(axis=1)
df['otono'] = df[['septiembre', 'octubre', 'noviembre']].sum(axis=1)
df['invierno'] = df[['diciembre', 'enero', 'febrero']].sum(axis=1)

df_estaciones = df[['region', 'primavera', 'verano', 'otono', 'invierno', 'anual']]
df_estaciones_pct = df_estaciones.set_index('region').apply(lambda x: x / x['anual'] * 100)
print("\nPorcentaje de precipitación por estación:")
print(df_estaciones_pct.head())


In [ ]:

# 9. Comparar precipitaciones entre regiones costeras e interiores
regiones_costeras = ['A CORUNA', 'ASTURIAS', 'CANTABRIA', 'BIZKAIA', 'GIPUZKOA', 'BARCELONA', 
                     'TARRAGONA', 'CASTELLON', 'VALENCIA', 'ALICANTE', 'MURCIA', 'ALMERIA', 
                     'GRANADA', 'MALAGA', 'CADIZ', 'HUELVA', 'PONTEVEDRA', 'ILLES BALEARS', 
                     'LAS PALMAS', 'SANTA CRUZ DE TENERIFE', 'CEUTA', 'MELILLA']

df['tipo_region'] = df['region'].apply(lambda x: 'Costera' if x in regiones_costeras else 'Interior')
comparacion = df.groupby('tipo_region')['anual'].agg(['mean', 'median', 'std'])
print("\nComparación de precipitaciones entre regiones costeras e interiores:")
print(comparacion)


In [ ]:

# 10. Identificar posibles anomalías
df_melt = df.melt(id_vars=['Parametro', 'region', 'anual'], 
                  var_name='mes', value_name='precipitacion')
df_melt['z_score'] = df_melt.groupby('mes')['precipitacion'].transform(
    lambda x: (x - x.mean()) / x.std()
)
anomalias = df_melt[df_melt['z_score'].abs() > 3]
print("\nPosibles anomalías en los datos de precipitación:")
print(anomalias[['region', 'mes', 'precipitacion', 'z_score']])

# Ejercicio de presentación del concepto 'Tidy Data'

Analicemos el siguiente archivo del Instituto de Meteorología e Hidrología de Panamá: https://www.imhpa.gob.pa/es/documentos, el archivo: 'Aportes Acumulados de los Embalses Bayano, Fortuna y Changuinola I'

Una base de datos tidy es una base de datos en la cuál
+ Cada variable que se medida debe estar en una columna.
+ Cada observación distinta de esa variable debe estar en una fila diferente.

In [ ]:
ruta = '/content/aportes_acumulados_de_los_embalses_bayanofortuna_y_changuinola-861562722.xls'

Paso 1: Leer el Archivo Excel

In [ ]:
df_embalses = pd.read_excel(ruta, header=None, sheet_name='Datos')
df_embalses.head()

Paso 2: Inspeccionar y Limpiar el DataFrame

In [ ]:
print(df_embalses.head(20))  # Muestra las primeras 20 filas para inspeccionar el formato

Paso 3: Separar y Renombrar las Columnas

In [ ]:
# Para Embalse Bayano
df_bayano = df_embalses.iloc[3:34, 0:6]
df_bayano.columns = ['Fecha', 'Historico (1976-2023)', 'Húmedo 2010', 'Seco 1976', '2023', '2024']
df_bayano['Embalse'] = 'Bayano'
df_bayano.head()

In [ ]:
# Para Embalse Fortuna
df_fortuna = df_embalses.iloc[3:34, 6:12]
df_fortuna.columns = ['Fecha', 'Historico (1994-2023)', 'Húmedo 1970', 'Seco 1983', '2023', '2024']
df_fortuna['Embalse'] = 'Fortuna'
df_fortuna.head()

In [ ]:
# Para Embalse Changuinola I
df_changuinola = df_embalses.iloc[3:34, 12:18]  # Ajusta el rango de columnas según sea necesario
df_changuinola.columns = ['Fecha', 'Historico (2012-2023)', 'Seco 2013', 'Húmedo 2008','2023','2024']
df_changuinola['Embalse'] = 'Changuinola I'
df_changuinola.head()

Paso 4: Transformamos el DataFrame para obtener un formato tidy para los años 2023 y 2024

In [ ]:
df_bayano_tidy = pd.melt(df_bayano,id_vars=['Fecha'], value_vars=['2023', '2024'],var_name='Categoria',value_name='Valor')
df_bayano_tidy


In [ ]:
df_bayano_tidy.info()

Podemos evidenciar que el formato de la columna valor, requiere una transformación de datos

In [ ]:
# Convertimos la columna 'Valor' a float
df_bayano_tidy['Valor'] = pd.to_numeric(df_bayano_tidy['Valor'], errors='coerce')

In [ ]:
# Ordenamos el DataFrame por Fecha y Categoria
df_bayano_tidy = df_bayano_tidy.sort_values(['Fecha', 'Categoria']).reset_index(drop=True)

In [ ]:
# Mostramos la información sobre el DataFrame para verificar el tipo de datos
print("Información del DataFrame:")
print(df_bayano_tidy.info())

In [ ]:
# Mostramos las primeras filas del DataFrame
print("\nPrimeras filas del DataFrame:")
print(df_bayano_tidy.head().round(2))


In [ ]:
# Mostramos las estadísticas descriptivas de la columna 'Valor'
print("\nEstadísticas descriptivas de la columna 'Valor':")
print(df_bayano_tidy['Valor'].describe().round(2))


In [ ]:
# Verificamos si hay valores nulos después de la conversión
print("\nNúmero de valores nulos en la columna 'Valor':")
print(df_bayano_tidy['Valor'].isnull().sum())

# Ejercicio: Replica el ejercicio para los 2 embalses restantes